In [2]:
import pandas as pd
import numpy as np
import geonamescache
from geonamescache.mappers import country

In [3]:
df = pd.read_csv('ds_salaries.csv')
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           3755 non-null   int64 
 1   experience_level    3755 non-null   object
 2   employment_type     3755 non-null   object
 3   job_title           3755 non-null   object
 4   salary              3755 non-null   int64 
 5   salary_currency     3755 non-null   object
 6   salary_in_usd       3755 non-null   int64 
 7   employee_residence  3755 non-null   object
 8   remote_ratio        3755 non-null   int64 
 9   company_location    3755 non-null   object
 10  company_size        3755 non-null   object
dtypes: int64(4), object(7)
memory usage: 322.8+ KB


In [5]:
for i in ['work_year', 'experience_level', 'employment_type', 'salary_currency', 'employee_residence', 'company_location', 'remote_ratio', 'job_title']:
    print(i)
    print(df[i].unique())

work_year
[2023 2022 2020 2021]
experience_level
['SE' 'MI' 'EN' 'EX']
employment_type
['FT' 'CT' 'FL' 'PT']
salary_currency
['EUR' 'USD' 'INR' 'HKD' 'CHF' 'GBP' 'AUD' 'SGD' 'CAD' 'ILS' 'BRL' 'THB'
 'PLN' 'HUF' 'CZK' 'DKK' 'JPY' 'MXN' 'TRY' 'CLP']
employee_residence
['ES' 'US' 'CA' 'DE' 'GB' 'NG' 'IN' 'HK' 'PT' 'NL' 'CH' 'CF' 'FR' 'AU'
 'FI' 'UA' 'IE' 'IL' 'GH' 'AT' 'CO' 'SG' 'SE' 'SI' 'MX' 'UZ' 'BR' 'TH'
 'HR' 'PL' 'KW' 'VN' 'CY' 'AR' 'AM' 'BA' 'KE' 'GR' 'MK' 'LV' 'RO' 'PK'
 'IT' 'MA' 'LT' 'BE' 'AS' 'IR' 'HU' 'SK' 'CN' 'CZ' 'CR' 'TR' 'CL' 'PR'
 'DK' 'BO' 'PH' 'DO' 'EG' 'ID' 'AE' 'MY' 'JP' 'EE' 'HN' 'TN' 'RU' 'DZ'
 'IQ' 'BG' 'JE' 'RS' 'NZ' 'MD' 'LU' 'MT']
company_location
['ES' 'US' 'CA' 'DE' 'GB' 'NG' 'IN' 'HK' 'NL' 'CH' 'CF' 'FR' 'FI' 'UA'
 'IE' 'IL' 'GH' 'CO' 'SG' 'AU' 'SE' 'SI' 'MX' 'BR' 'PT' 'RU' 'TH' 'HR'
 'VN' 'EE' 'AM' 'BA' 'KE' 'GR' 'MK' 'LV' 'RO' 'PK' 'IT' 'MA' 'PL' 'AL'
 'AR' 'LT' 'AS' 'CR' 'IR' 'BS' 'HU' 'AT' 'SK' 'CZ' 'TR' 'PR' 'DK' 'BO'
 'PH' 'BE' 'ID' 'EG' 'AE' 'LU' 'MY'

In [6]:
df.isna().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [7]:
df.duplicated().sum()

# karena tidak ada Primary Key maka duplicated akan dibiarkan

np.int64(1171)

In [8]:
mapper = country(from_key='iso', to_key='name')

def get_country_name(df_string, mapper=mapper):
    result = mapper(df_string)
    
    # if country ID is not found, set result to invalid
    if result == None: 
        result = 'Invalid'
    
    return result

In [9]:
employee_res = df['employee_residence'][:].apply(get_country_name, mapper=mapper)
employee_res.value_counts()



employee_residence
United States     3004
United Kingdom     167
Canada              85
Spain               80
India               71
                  ... 
Serbia               1
New Zealand          1
Moldova              1
Luxembourg           1
Malta                1
Name: count, Length: 78, dtype: int64

In [10]:
df['employee_residence'] = employee_res
mask = df['employee_residence'] == 'Invalid'
df[mask].shape[0]

0

In [11]:
company_loc = df['company_location'][:].apply(get_country_name, mapper=mapper)
company_loc.value_counts()

company_location
United States     3040
United Kingdom     172
Canada              87
Spain               77
India               58
                  ... 
China                1
New Zealand          1
Chile                1
Moldova              1
Malta                1
Name: count, Length: 72, dtype: int64

In [12]:
df['company_location'] = company_loc
mask = df['company_location'] == 'Invalid'
df[mask].shape[0]

0

In [13]:
df['company_size'].unique()

array(['L', 'S', 'M'], dtype=object)

In [14]:
# Mengubah kolom employment_type
def emp_type(i):
    if i == 'FT':
        return 'Full Time'
    elif i == 'CT':
        return 'Contract'
    elif i == 'FL':
        return 'Freelance'
    else:
        return 'Part Time'

# Menerapkan fungsi ke kolom employment_type
df['employment_type'] = df['employment_type'].apply(emp_type)

print(df['employment_type'])

0       Full Time
1        Contract
2        Contract
3       Full Time
4       Full Time
          ...    
3750    Full Time
3751    Full Time
3752    Full Time
3753     Contract
3754    Full Time
Name: employment_type, Length: 3755, dtype: object


In [15]:
# Mengubah kolom experience_level
def exp_lvl(i):
    if i == 'SE':
        return 'Senior Level'
    elif i == 'MI':
        return 'Mid Level'
    elif i == 'EX':
        return 'Experience'
    else:
        return 'Entry Level'

# Menerapkan fungsi ke kolom experience_level
df['experience_level'] = df['experience_level'].apply(exp_lvl)

print(df['experience_level'])

0       Senior Level
1          Mid Level
2          Mid Level
3       Senior Level
4       Senior Level
            ...     
3750    Senior Level
3751       Mid Level
3752     Entry Level
3753     Entry Level
3754    Senior Level
Name: experience_level, Length: 3755, dtype: object


In [16]:
# Mengubah kolom remote_ratio
def remote(i):
    if i == 100:
        return 'WFH'
    elif i == 50:
        return 'Hybrid'
    else:
        return 'WFO'

# Menerapkan fungsi ke kolom experience_level
df['remote_ratio'] = df['remote_ratio'].apply(remote)

print(df['remote_ratio'])

0          WFH
1          WFH
2          WFH
3          WFH
4          WFH
         ...  
3750       WFH
3751       WFH
3752       WFH
3753       WFH
3754    Hybrid
Name: remote_ratio, Length: 3755, dtype: object


In [17]:
df['company_size'] = df['company_size'].replace('L', 'Large')
df['company_size'] = df['company_size'].replace('M', 'Medium')
df['company_size'] = df['company_size'].replace('S', 'Small')

In [18]:
df['job_title']

0       Principal Data Scientist
1                    ML Engineer
2                    ML Engineer
3                 Data Scientist
4                 Data Scientist
                  ...           
3750              Data Scientist
3751    Principal Data Scientist
3752              Data Scientist
3753       Business Data Analyst
3754        Data Science Manager
Name: job_title, Length: 3755, dtype: object

In [19]:
filtered_nama = df[df['job_title'].str.contains('Scien', na=False)]['job_title'].unique()

filtered_nama

array(['Principal Data Scientist', 'Data Scientist', 'Applied Scientist',
       'Research Scientist', 'Applied Data Scientist',
       'Data Science Manager', 'Director of Data Science',
       'Machine Learning Scientist', 'AI Scientist',
       'Applied Machine Learning Scientist', 'Lead Data Scientist',
       'Data Science Lead', 'Data Science Consultant',
       'Head of Data Science', 'Data Science Engineer',
       'Data Science Tech Lead', 'Data Scientist Lead',
       'Product Data Scientist', 'Staff Data Scientist'], dtype=object)

In [20]:
# Fungsi untuk mengkategorikan pekerjaan
def job_cat(i):
    if 'anal' in i.lower():
        return 'Data Analyst'
    elif 'scien' in i.lower():
        return 'Data Scientist'
    else:
        return 'Data Engineer'

# Menerapkan fungsi ke kolom job_title
df['job_kat'] = df['job_title'].apply(job_cat)


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           3755 non-null   int64 
 1   experience_level    3755 non-null   object
 2   employment_type     3755 non-null   object
 3   job_title           3755 non-null   object
 4   salary              3755 non-null   int64 
 5   salary_currency     3755 non-null   object
 6   salary_in_usd       3755 non-null   int64 
 7   employee_residence  3755 non-null   object
 8   remote_ratio        3755 non-null   object
 9   company_location    3755 non-null   object
 10  company_size        3755 non-null   object
 11  job_kat             3755 non-null   object
dtypes: int64(3), object(9)
memory usage: 352.2+ KB


In [22]:
filtered_df = df[df['job_kat'] == 'Data Engineer']
unique_job_titles = filtered_df['job_title'].unique()

unique_job_titles

array(['ML Engineer', 'Data Modeler', 'Research Engineer',
       'Business Intelligence Engineer', 'Machine Learning Engineer',
       'Data Strategist', 'Data Engineer', 'Computer Vision Engineer',
       'Data Architect', 'Applied Machine Learning Engineer',
       'AI Developer', 'ETL Engineer', 'Data DevOps Engineer',
       'Head of Data', 'Data Manager', 'Machine Learning Researcher',
       'Big Data Engineer', 'Data Specialist', 'BI Data Engineer',
       'MLOps Engineer', 'Autonomous Vehicle Technician',
       'Cloud Database Engineer', 'Data Infrastructure Engineer',
       'Software Data Engineer', 'AI Programmer',
       'Data Operations Engineer', 'BI Developer',
       'Deep Learning Researcher',
       'Machine Learning Infrastructure Engineer',
       'Deep Learning Engineer', 'Machine Learning Software Engineer',
       'Big Data Architect', 'Computer Vision Software Engineer',
       'Azure Data Engineer', 'Marketing Data Engineer', 'Data Lead',
       'Machine Lear

In [23]:
# Kolom Harga
print(f'Mean.salary     : {df["salary"].mean().round(2)}')
print(f'Median.salary   : {df["salary"].median()}')
print(f'St_Dev.salary   : {df["salary"].std().round(2)}')
print(f'Skewness.salary : {df["salary"].skew().round(2)}')
print(f'Kurtosis.salary : {df["salary"].kurtosis().round(2)}')
print(f"salary Min.    : {df['salary'].min()}")
print(f"salary Max.    : {df['salary'].max()}")

Mean.salary     : 190695.57
Median.salary   : 138000.0
St_Dev.salary   : 671676.5
Skewness.salary : 28.94
Kurtosis.salary : 1147.57
salary Min.    : 6000
salary Max.    : 30400000


In [24]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,job_kat
0,2023,Senior Level,Full Time,Principal Data Scientist,80000,EUR,85847,Spain,WFH,Spain,Large,Data Scientist
1,2023,Mid Level,Contract,ML Engineer,30000,USD,30000,United States,WFH,United States,Small,Data Engineer
2,2023,Mid Level,Contract,ML Engineer,25500,USD,25500,United States,WFH,United States,Small,Data Engineer
3,2023,Senior Level,Full Time,Data Scientist,175000,USD,175000,Canada,WFH,Canada,Medium,Data Scientist
4,2023,Senior Level,Full Time,Data Scientist,120000,USD,120000,Canada,WFH,Canada,Medium,Data Scientist


In [25]:
pivot_table = df.pivot_table(values='salary', index='job_kat', columns='work_year', aggfunc='count')

In [27]:
filtered_df = df[df['work_year'].isin([2022, 2023])]
pivot_avg = filtered_df.pivot_table(values='salary_in_usd', index='job_kat', columns='work_year', aggfunc='count')
pivot_avg = pivot_avg.round(2)


pivot_avg['persen'] = ((pivot_avg[2023] - pivot_avg[2022]) / pivot_avg[2022]) * 100
pivot_avg['persen'] = pivot_avg['persen'].round(2)

pivot_avg

work_year,2022,2023,persen
job_kat,,,
Data Analyst,377,390,3.45
Data Engineer,759,869,14.49
Data Scientist,528,526,-0.38


Pengaruh Tahun Kerja Terhadap Gaji:

Bagaimana perubahan tahun kerja (work_year) mempengaruhi rata-rata gaji (salary_in_usd) di berbagai kategori pekerjaan?
Pengaruh Tingkat Pengalaman Terhadap Gaji:

Apa hubungan antara tingkat pengalaman (experience_level) dan gaji? Apakah ada perbedaan signifikan antara level pemula, menengah, dan senior?
Perbandingan Gaji Berdasarkan Tipe Pekerjaan:

Bagaimana gaji bervariasi berdasarkan jenis pekerjaan (employment_type), seperti penuh waktu, paruh waktu, atau kontrak?
Dampak Lokasi Perusahaan Terhadap Gaji:

Bagaimana lokasi perusahaan (company_location) mempengaruhi gaji? Apakah ada perbedaan signifikan antara lokasi yang berbeda, seperti kota besar vs. kota kecil?
Gaji dan Rasio Remote:

Apakah ada hubungan antara rasio remote (remote_ratio) dan gaji? Apakah pekerja yang bekerja dari jarak jauh mendapatkan gaji yang lebih tinggi atau lebih rendah dibandingkan dengan yang bekerja di lokasi?
Analisis Gaji Berdasarkan Ukuran Perusahaan:

Bagaimana ukuran perusahaan (company_size) mempengaruhi gaji? Apakah karyawan di perusahaan besar mendapatkan gaji yang lebih tinggi dibandingkan di perusahaan kecil?
Perbandingan Gaji Berdasarkan Kategori Pekerjaan:

Bagaimana gaji bervariasi berdasarkan kategori pekerjaan (job_category)? Kategori pekerjaan mana yang memiliki gaji tertinggi dan terendah?
Tren Gaji dari Tahun ke Tahun:

Bagaimana tren gaji berubah dari tahun ke tahun untuk berbagai kategori pekerjaan? Apakah gaji meningkat, menurun, atau stabil?
Analisis Keterkaitan Antara Lokasi dan Tipe Pekerjaan:

Apakah ada pola tertentu di mana jenis pekerjaan lebih banyak ditemukan di lokasi tertentu? Bagaimana hal ini mempengaruhi gaji?

In [ ]:
# Memfilter rata-rata gaji hanya 'Data Analyst' saja
filtered_df = df[(df['job_category'] == 'Data Scientist')]
# filtered_df = df[(df['work_year'] == 2023) & (df['job_category'] == 'Data Scientist')]
pvt_ds = filtered_df.pivot_table(values='salary_in_usd', columns='job_category', aggfunc='mean')

# Menampilkan hasil dalam bentuk DataFrame
pvt_ds = pd.DataFrame(pvt_ds.round(2))
pvt_ds